In [1]:
from gui.Main import Main
main=Main()
import logging

import sqlalchemy_dao
from sqlalchemy_dao import Dao

from conf.ConfigReader import ConfigReader
from db.ORMs import Filter
from gui.Workflow import Workflow
from utils.AnnotationTypeDef import AnnotationTypeDef
from utils.IntroStep import IntroStep
from utils.KeywordsFiltering import KeywordsFiltering
from utils.KeywordsUMLSExtender import KeywordsUMLSExtender
from utils.KeywordsUMLSExtenderSetup import KeywordsUMLSExtenderSetup
from gui.PreviousNextWidgets import PreviousNextHTML
from utils.ReviewRBInit import ReviewRBInit
from utils.ReviewRBLoop import ReviewRBLoop
from utils.ReviewMLInit import ReviewMLInit
from utils.ReviewMLLoop import ReviewMLLoop
from models.logistic.LogisticBOWClassifier import LogisticBOWClassifier
from utils.DataSetChooser import DataSetChooser

logging.getLogger().setLevel(logging.DEBUG)

ConfigReader('../conf/smartanno_conf.json')

wf = Workflow(config_file=ConfigReader.config_file)
wf.api_key = ConfigReader.getValue("api_key")
wf.dao = Dao('sqlite+pysqlite:///../data/test.sqlite', sqlalchemy_dao.POOL_DISABLED)
wf.task_name = 'language'
wf.append(DataSetChooser(name='dataset_chooser',description='<h4>Choose which dateaset you want to use: </h4>'))
wf.append(AnnotationTypeDef(
    '<h3>Annotation types:</h3><p>List all the types you want to identify below. Each type per line.<br/>If you'
    'have too many types, try set up them separately, so that you won&apos;t need to choose from a long list '
    'for each sample. </p>', name='types'))
wf.append(KeywordsFiltering(
    name='keywords'))

rb=ReviewRBInit(name="rb_review_init")
wf.append(rb)
rv = ReviewRBLoop(name='rb_review', rush_rule='../conf/rush_rules.tsv')
wf.append(rv)
wf.append(PreviousNextHTML('<h2>Congratuations!</h2><h4>You have finished the initial review on the rule-base preannotations. </h4>',
                       name='intro'))
wf.append(ReviewMLInit(name='ml_review_init'))
wf.append(ReviewMLLoop(name='ml_review',ml_classifier_cls=LogisticBOWClassifier))

wf.start()
wf.steps[0].complete()
wf.steps[1].complete()
wf.steps[2].complete()

VBox(children=(IntProgress(value=1, description='Progress: 1/15', layout=Layout(height='14px', width='90%'), max=15, min=1, style=ProgressStyle(description_width='initial')), HTML(value='<table style="width:100%"><tr><th>DOC_ID</th><th>DATE</th><th>REF_DATE</th></tr><tr><td>41467</td><td>2091-08-24</td><td>2093-07-10</td></tr></table><div id="d1" style="overflow-y: scroll; height:200px;border:1px solid;border-color:#e5e8e8; ">APTT                   26.4                      22.1-35.1      sec</div>'), ClickResponsiveToggleButtons(button_style='success', index=2, layout=Layout(width='70%'), options=('Eng', 'NotEng', 'Irrelevant'), value='Irrelevant'), Label(value='', layout=Layout(height='10px')), IntProgress(value=1, layout=Layout(height='10px', left='1%', width='98%'), max=1, style=ProgressStyle(bar_color='GAINSBORO')), Label(value='', layout=Layout(height='1px')), HBox(children=(Button(description='Previous', layout=Layout(left='100px', width='90px'), style=ButtonStyle()), Button(description='Next', layout=Layout(left='100px', width='90px'), style=ButtonStyle()), Button(description='Complete', layout=Layout(left='100px', width='90px'), style=ButtonStyle())), layout=Layout(left='10%', width='80%'))))

In [3]:
wf.steps[3].sample_sizes

{'Eng': 4, 'Irrelevant': 5, 'NotEng': 5, 'not_contain': 6}

In [2]:
sampler=wf.steps[3].sampler

In [3]:
list=[len(sampler.all_contain_ids), 
      len(sampler.previous_sampled_ids),
     sampler.available_not_contain,
     ]
list

[2746, 133, 58712]

In [4]:
{type_name:len(ids) for type_name,ids in sampler.new_ids.items()}

{'Eng': 52, 'Irrelevant': 2599, 'NotEng': 19}

In [5]:
{type_name:len(ids)-len(sampler.new_ids[type_name]) for type_name,ids in sampler.grouped_ids.items()}

{'Eng': 28, 'Irrelevant': 29, 'NotEng': 19}

In [6]:
len(sampler.all_contain_ids.intersection(sampler.previous_sampled_ids))

76

In [7]:
import ipywidgets as widget

In [8]:
wf.steps[1].start()

VBox(children=(HTML(value='<h3>Annotation types:</h3><p>List all the types you want to identify below. Each type per line.<br/>If youhave too many types, try set up them separately, so that you won&apos;t need to choose from a long list for each sample. </p>'), Textarea(value='Eng\nNotEng\nIrrelevant', layout=Layout(height='300px', width='500px'), placeholder='each phrase/word per line'), Label(value='', layout=Layout(height='10px')), IntProgress(value=1, layout=Layout(height='10px', left='1%', width='98%'), max=1, style=ProgressStyle(bar_color='GAINSBORO')), Label(value='', layout=Layout(height='1px')), HBox(children=(Button(description='Previous', layout=Layout(width='90px'), style=ButtonStyle(button_color='SILVER')), Button(description='Next', layout=Layout(left='100px', width='90px'), style=ButtonStyle(button_color='SANDYBROWN'))), layout=Layout(left='10%', width='80%'))), layout=Layout(display='flex'))